# Quantum Process Tomography

* **Last Updated:** Feb 20, 2019
* **Requires:** qiskit-terra 0.7

In [3]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

## 1-qubit process tomography example

In [5]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'0': 2013, '1': 1987},
 (('Zp',), ('Z',)): {'0': 1970, '1': 2030},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'0': 2016, '1': 1984},
 (('Zm',), ('Z',)): {'0': 1989, '1': 2011},
 (('Xp',), ('X',)): {'0': 2025, '1': 1975},
 (('Xp',), ('Y',)): {'0': 1972, '1': 2028},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'0': 2062, '1': 1938},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'0': 1981, '1': 2019}}

In [6]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

Least-Sq Fitter
fit time: 0.014121055603027344
fit fidelity: 0.9974445877456369

CVXOPT Fitter
fit time: 0.0778050422668457
fit fidelity: 0.9999423401755929


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [8]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 989, '1': 1011},
 (('Zp',), ('Y',)): {'0': 977, '1': 1023},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 1017, '1': 983},
 (('Zm',), ('Y',)): {'0': 1054, '1': 946},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 1003, '1': 997},
 (('Xp',), ('Z',)): {'0': 1000, '1': 1000},
 (('Yp',), ('X',)): {'0': 954, '1': 1046},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 1003, '1': 997}}

In [11]:
# Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

Least-Sq Fitter
fit time: 0.008608102798461914
fit fidelity: 0.9888224465913296

CVXOPT Fitter
fit time: 0.04039311408996582
fit fidelity: 0.9997691179639802


## 2-Qubit entangling circuit

In [13]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)

t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_lstsq / 4))

t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

Least-Sq Fitter
fit time: 0.11325693130493164
fit fidelity: 0.9875860200666599

CVXOPT Fitter
fit time: 1.2218871116638184
fit fidelity: 0.9999038654777672


## Using SIC-POVM preparation basis

In [16]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs, prep_basis='SIC')
qpt_tomo.data

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'0': 1026, '1': 974},
 (('S0',), ('Z',)): {'0': 962, '1': 1038},
 (('S1',), ('X',)): {'0': 635, '1': 1365},
 (('S1',), ('Y',)): {'0': 978, '1': 1022},
 (('S1',), ('Z',)): {'0': 1943, '1': 57},
 (('S2',), ('X',)): {'0': 676, '1': 1324},
 (('S2',), ('Y',)): {'0': 1805, '1': 195},
 (('S2',), ('Z',)): {'0': 548, '1': 1452},
 (('S3',), ('X',)): {'0': 707, '1': 1293},
 (('S3',), ('Y',)): {'0': 177, '1': 1823},
 (('S3',), ('Z',)): {'0': 537, '1': 1463}}

In [17]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_lstsq / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

Least-Sq Fitter
fit time: 0.009093284606933594
fit fidelity: 0.9877662037111621

CVXOPT Fitter
fit time: 0.05207705497741699
fit fidelity: 0.9947304741419135
